In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute,QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, array_to_latex
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction

from qiskit.circuit.library import QFT
from numpy import pi
from scipy.signal import find_peaks
from time import time

import warnings
warnings.filterwarnings('ignore')


# the circuit for the unitary of interest
U = QuantumCircuit(8)        
U.swap(0,1)
U.swap(2,3)
U.barrier()
U.swap(1,2)
U.swap(3,4)
U.barrier()
U.swap(2,3)
U.swap(4,5)
U.barrier()
U.swap(3,4)
U.swap(5,6)
U.barrier()
U.swap(4,5)
U.swap(6,7)
U.barrier() 
U.swap(5,6)
U.swap(7,0)
U.barrier() 

# the power of a unitary
def circ_rep(k,circ):
    num_qubits= circ.num_qubits
    U_rep=QuantumCircuit(num_qubits)
    for i in range(k):
        U_rep=U_rep.compose(U)
    
    return U_rep

# device a circuit into cycles
def divide_circ(circ):
    circ_list=[[]]
    ind=0
    for ins in circ[0:-1]:
        if 'barrier' == ins[0].name:
            ind=ind+1
            circ_list.append([])
        else:
            circ_list[ind].append(ins)
    return circ_list          

# the product of Pauli opeartors
def prod_pauli(plist):
    pauli_list=np.array([[0,0],[0,1],[1,0],[1,1]])
    s=np.array([0,0])
    for q in plist:
        s=s+pauli_list[q]
    return [i%2 for i in s]

# randomized comping on a cycle
def compiled_cycle(bare_cycle,cor_paulis0,nq=8):
    g1_list=[]
    g2_list=[]
    for ins in bare_cycle:
        if ins[0].num_qubits==1:
            g1_list.append(2)
            
        else:
            g2_list.append(ins)
            
    rand_paulis=np.random.randint(4,size=nq)
    compiled_ops=[]
    
    if g1_list==[]:
        g1_list=[0 for i in range(nq)]
    for q in range(nq):
        compiled_ops.append(prod_pauli([rand_paulis[q],g1_list[q],cor_paulis0[q]]))
    
    cor_paulis1=rand_paulis.copy()
    ind1=ind2=0
    
    for ins in g2_list:
        ind1, ind2= (qubit.index for qubit in ins[1])
        
        cor_paulis1[ind1],cor_paulis1[ind2]= cor_paulis1[ind2],cor_paulis1[ind1]
    return compiled_ops,g2_list,cor_paulis1

# define Pauli gates on circuit
class myqc(QuantumCircuit):
    def set_ct(self,cq,tq):
        self.cq=cq
        self.tq=tq
        
    def paulis(self,qi,x):  
        if x==[0,0]:
            self.id(qi)
                     
        elif x==[0,1]:
            self.z(qi)
                     
        elif x==[1,0]:
            self.x(qi)
        elif x==[1,1]:
            self.y(qi)

# randomized compiling on a circuit
def randomized_compiling(bare_circ,nq=8):
    circ1=divide_circ(bare_circ)
    
    qc=myqc(nq)
    cor_paulis0= [0 for i in range(nq)]
    for cycle in circ1:
        compiled_ops,g2_list,cor_paulis1=compiled_cycle(cycle,cor_paulis0,nq)
        
        for i in range(nq):
            qc.paulis(i,compiled_ops[i])
        if g2_list!=[]:
            g2_num= len(g2_list)
            for i in range(g2_num):
                qc.append(*g2_list[i])
        
        qc.barrier()
        cor_paulis0= cor_paulis1
    #print(cor_paulis0)    
    pauli_list=[[0,0],[0,1],[1,0],[1,1]]
    
    for i in range(nq):
        qc.paulis(i,pauli_list[cor_paulis0[i]])
    
    return qc

# generate the full circuit for simulation with initial state and final measurement
def full_circ(floq_circ,mode='x',nq=8):
    qc= QuantumCircuit(nq,1)
    qc.h(nq-1)
    qc.barrier()
    
    qc=qc.compose(floq_circ)
    if mode=='y':
        qc.s(nq-1)
    
    qc.h(nq-1)
    qc.barrier()
    qc.measure(nq-1,0)
    
    return qc

# noise model for simulation
from qiskit.providers.aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, coherent_unitary_error, mixed_unitary_error
from scipy.optimize import curve_fit, minimize

def unitary_error_rx(theta):  #exp(-i theta/2 X)
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], [-1j*np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_rxx(theta):          #exp(-i theta/2 X\otimes X)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]])

def unitary_error_rz(theta):  #exp(-i theta/2 Z)
    return np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

def unitary_error_rzz(theta):          #exp(-i theta/2 X\otimes X)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

def unitary_error_ry(theta):  #exp(-i theta/2 Y)
    return np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_ryy(theta):          #exp(-i theta/2 Y\otimes Y)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]])

# generate stochastic model
def get_noise_model(p=0,mode='sy'):
    noise_model=NoiseModel()
    
    if mode=='sx':
        mu_1q=mixed_unitary_error([(1/np.sqrt(2)*np.array([[1,-1j],[-1j,1]]),p/2),(np.eye(2),1-p/2)])
        mu_2q= mu_1q.tensor(mu_1q)
    elif mode=='sy':
        mu_1q=mixed_unitary_error([(1/np.sqrt(2)*np.array([[1,-1],[1,1]]),p/2),(np.eye(2),1-p/2)])
        mu_2q= mu_1q.tensor(mu_1q)
    elif mode=='sz':
        mu_1q=mixed_unitary_error([(np.array([[1,0],[0,1j]]),p/2),(np.eye(2),1-p/2)])
        mu_2q= mu_1q.tensor(mu_1q)
    
    qerror_1q = mu_1q
    qerror_2q= mu_2q

    noise_model.add_all_qubit_quantum_error(qerror_1q, ['u2','u3','I','X','Y','Z'])
    noise_model.add_all_qubit_quantum_error(qerror_2q, ['swap'])
    
    return noise_model

# simulation with noise 
def noise_sim(circ_list,noise_model,shots=8192):
    result = execute(circ_list, backend=Aer.get_backend('qasm_simulator'),noise_model=noise_model, shots=shots,optimization_level=0).result()
    count_list=[result.get_counts(circ) for circ in circ_list]
    exp_list=[2*count.get('0',0)/shots-1 for count in count_list]
    
    return exp_list


In [ ]:
# generate bare circuits for simulation
circ_rep_list=[circ_rep(k,U) for k in range(1,101)]
full_circ_mx_list=list(map(full_circ,circ_rep_list))
full_circ_my_list=list(map(lambda x:full_circ(x,mode='y'),circ_rep_list))

In [ ]:
# simulation with bare circuit
noise_mode='sy'
noise_pro= 0.1
noise_model= get_noise_model(noise_pro,noise_mode)
xlist=noise_sim(full_circ_mx_list,noise_model=noise_model,shots=10**5)
ylist=noise_sim(full_circ_my_list,noise_model=noise_model,shots=10**5)
data_list= np.array(xlist)+1j*np.array(ylist)
fourier_amp=list(np.absolute(np.fft.fft(data_list)))
plt.plot(fourier_amp)

In [ ]:
# generate randomized circuits
ts= time()
cq_list=[]
for qc in circ_rep_list:
    cq_list.append([randomized_compiling(qc) for i in range(20)])
te= time()
print(te-ts)

In [ ]:
# simulation with randomized circuits
shots_tot= 10**5
nrand= 20  # the num of randomized circuits for each bare circuit
shots= shots_tot/nrand

xlist=[]
for cqs in cq_list:
    cqs_full=list(map(full_circ,cqs[0:nrand]))
    xlist.append(np.mean(noise_sim(cqs_full,noise_model,shots=shots)))

ylist=[]
for cqs in cq_list:
    cqs_full=list(map(lambda x:full_circ(x,mode='y'),cqs[0:nrand]))
    ylist.append(np.mean(noise_sim(cqs_full,noise_model,shots=shots)))

data_list= np.array(xlist)+1j*np.array(ylist)
fourier_amp=list(np.absolute(np.fft.fft(data_list)))
plt.plot(fourier_amp)
